# Delaunay Tesselation Field Estimator in 3D

In [11]:
#Load the numpy and scipy libraries
import numpy as np
from scipy.spatial import Delaunay

#The Delaunay Tesselation Field Estimator 
class DTFE:
    def __init__(self, points, velocities, m):
        print("Delaunay Tesselation Field Estimator initialization:")
        self.velocities = velocities
        print("\t-Evaluate Delaunay tessellation")
        self.delaunay = Delaunay(points)
        
        #Area of a triangle
        def volume(sim, points):
            return abs(np.linalg.det(np.array([points[sim[1]] - points[sim[0]], 
                                               points[sim[2]] - points[sim[0]],
                                               points[sim[3]] - points[sim[0]]]))) / 6
        
        #The density estimate
        print("\t-Evaluate density estimate")
        self.rho = np.zeros(len(points))
        for sim in self.delaunay.simplices:
            vol = volume(sim, points)
            for index in sim:
                self.rho[index] += vol
        self.rho = (3 + 1) * m / self.rho
        
        #The gradients
        print("\t-Evaluate gradients")
        self.Drho = np.zeros([len(self.delaunay.simplices), 3])
        self.Dv   = np.zeros([len(self.delaunay.simplices), 3, 3])
        for i in np.arange(len(self.delaunay.simplices)):
            [p0, p1, p2, p3] = points[self.delaunay.simplices[i]] 
            [r0, r1, r2, r3] = self.rho[self.delaunay.simplices[i]]
            [v0, v1, v2, v3] = self.velocities[self.delaunay.simplices[i]]
        
            A = np.array([p1 - p0, p2 - p0, p3 - p0])
            self.Drho[i] = np.linalg.inv(A) @ np.array([r1 - r0, r2 - r0, r3 - r0])
            self.Dv[i] = np.linalg.inv(A) @ np.array([v1 - v0, v2 - v0, v3 - v0])

    #The interpolations
    def density(self, x, y, z):
        simplexIndex = self.delaunay.find_simplex([x, y, z])
        pointIndex   = self.delaunay.simplices[simplexIndex][0]
        return self.rho[pointIndex] + self.Drho[simplexIndex] @ ([x,y, z] - self.delaunay.points[pointIndex])

    def v(self, x, y, z):
        simplexIndex = self.delaunay.find_simplex([x, y, z])
        pointIndex   = self.delaunay.simplices[simplexIndex][0]
        return self.velocities[pointIndex] + self.Dv[simplexIndex] @ ([x, y, z] - self.delaunay.points[pointIndex])

    def theta(self, x, y, z):
        simplexIndex = self.delaunay.find_simplex([x, y, z])
        pointIndex   = self.delaunay.simplices[simplexIndex][0]
        return self.Dv[simplexIndex][0,0] + self.Dv[simplexIndex][1,1] + self.Dv[simplexIndex][2,2]

    def sigma(self, x, y, z):
        simplexIndex = self.delaunay.find_simplex([x, y, z])
        pointIndex   = self.delaunay.simplices[simplexIndex][0]
        return np.array([[ self.Dv[simplexIndex][0,0], 
                         (self.Dv[simplexIndex][0,1] + self.Dv[simplexIndex][1,0]) / 2,
                         (self.Dv[simplexIndex][0,2] + self.Dv[simplexIndex][2,0]) / 2],
                        [(self.Dv[simplexIndex][1,0] + self.Dv[simplexIndex][0,1]) / 2, 
                          self.Dv[simplexIndex][1,1],
                         (self.Dv[simplexIndex][1,2] + self.Dv[simplexIndex][2,1]) / 2],
                        [(self.Dv[simplexIndex][2,0] + self.Dv[simplexIndex][0,2]) / 2,
                         (self.Dv[simplexIndex][2,1] + self.Dv[simplexIndex][1,2]) / 2,
                          self.Dv[simplexIndex][2,2]]]) - np.eye(3) * (self.Dv[simplexIndex][0,0] + self.Dv[simplexIndex][1,1] + self.Dv[simplexIndex][2,2])
    
    def omega(self, x, y, z):
        simplexIndex = self.delaunay.find_simplex([x, y, z])
        pointIndex   = self.delaunay.simplices[simplexIndex][0]
        return np.array([[0, (self.Dv[simplexIndex][0,1] - self.Dv[simplexIndex][1,0]) / 2, (self.Dv[simplexIndex][0,2] - self.Dv[simplexIndex][2,0]) / 2],
                        [(self.Dv[simplexIndex][1,0] - self.Dv[simplexIndex][0,1]) / 2, 0, (self.Dv[simplexIndex][1,2] - self.Dv[simplexIndex][2,1]) / 2],
                        [(self.Dv[simplexIndex][3,0] - self.Dv[simplexIndex][0,3]) / 2, (self.Dv[simplexIndex][2,1] - self.Dv[simplexIndex][1,2]) / 2, 0]]) 